In [1]:
import numpy as np
from scipy.linalg import expm, sqrtm, inv, pinv
import matplotlib.pyplot as plt

from qutip import *

import sys

In [2]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2, EstimatorV2, Estimator
from qiskit.transpiler import generate_preset_pass_manager, PassManager
from qiskit import transpile
from qiskit_aer import AerSimulator

from qiskit_experiments.library import StateTomography

In [3]:
service = QiskitRuntimeService() 

backend = service.least_busy(simulator=False, operational=True, filters=lambda b: "dynamic_circuits" in b.configuration().supported_features)
# backend = service.least_busy(simulator=False, operational=True)
backend

/tmp/ipykernel_18324/3621571705.py:1: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud" and "local" will be the only valid channels. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService()


<IBMBackend('ibm_brisbane')>

In [4]:
# Constants

I = np.eye(2)

In [5]:
# "Check" helper functions

def check_trace(rho: Qobj):
    """
    Check if the trace of the density matrix is 1.
    """
    return np.isclose(rho.tr(), 1.0, atol=1e-7)

def check_hermitian(H):
    """
    Check if the operator is Hermitian.
    """
    if isinstance(H, Qobj):
        return np.allclose(H.full(), H.dag().full(), atol=1e-10)
    else:
        return np.allclose(H, H.conjugate().T, atol=1e-10)

def check_unitary(U):
    """
    Check if the operator is unitary.
    """
    return np.allclose(U @ U.conjugate().T, I, atol=1e-6) and np.allclose(U.conjugate().T @ U, I, atol=1e-6)

def check_psd(mat):
    """
    Check if the matrix is positive semi-definite.
    """
    return np.all(np.linalg.eigvals(mat) >= -1e-6)

def check_choi_decomposition(choi_matrix, pos_choi, neg_choi):
    """
    Check if the Choi matrix is decomposed correctly.
    """
    return np.allclose(choi_matrix.full(), (pos_choi - neg_choi).full(), atol=1e-7)

def check_TP(kraus_operators, tol = 1e-7):
    """
    Check if the Kraus operators are trace-preserving.
    """
    if isinstance(kraus_operators[0], Qobj):    
        M = sum([k.dag() * k for k in kraus_operators])
        isclose = np.allclose(M.full(), I, atol=1e-7)
    else:
        M = sum([k.conjugate().T @ k for k in kraus_operators])
        isclose = np.allclose(M, I, atol= tol)

    if not isclose:
        print(M)
        print("Deviation from TP condition: ", np.linalg.norm(M - I))

    return isclose

In [6]:
# "Compute" helper functions

def compute_fidelity(rho1: Qobj, rho2: Qobj):
    """
    Calculate the fidelity between two density matrices.
    """
    return (rho1 * rho2).tr().sqrt()

def compute_expectation_value(rho:Qobj, A):
    """
    Calculate the expectation value of an operator A with respect to a density matrix rho.
    """
    return (rho * A).tr()

In [7]:
def choi_adjointchoi_transform(choi_matrix):
    """
    Convert the Choi matrix to its adjoint form, and vice versa (As defined in the supplementary material of the paper).
    """
    if isinstance(choi_matrix, Qobj):
        U_23 = Qobj(np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]))
        adjoint_choi = choi_matrix.transform(U_23)
        adjoint_choi = adjoint_choi.conj()
    else:
        U_23 = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
        adjoint_choi = U_23 @ choi_matrix @ U_23
        adjoint_choi = adjoint_choi.conjugate()
    
    return adjoint_choi

def construct_choi_matrix(map):
    """
    Construct the Choi matrix for the map.
    """

    choi_matrix = to_choi(map)

    # FIXED ERROR: The Choi matrix is not hermitian
    choi_matrix = (choi_matrix + choi_matrix.dag())/2

    return choi_matrix
    

def positive_and_negative_eigenspaces(choi_matrix):
    """
    Compute the positive and negative eigenspaces of the Choi matrix
    """
    # eigenvalues, eigenvectors = np.linalg.eigh(choi_matrix)
    eigenvalues = choi_matrix.eigenstates()[0]
    eigenvectors = choi_matrix.eigenstates()[1]

    # Positive eigenspace
    pos_eigenvalues = eigenvalues[eigenvalues > 0]
    pos_eigenvectors = eigenvectors[eigenvalues > 0]
    
    # Negative eigenspace
    neg_eigenvalues = (-1)*eigenvalues[eigenvalues < 0]
    neg_eigenvectors = eigenvectors[eigenvalues < 0]
    
    return pos_eigenvalues, pos_eigenvectors, neg_eigenvalues, neg_eigenvectors


def construct_choi_matrix_from_eigenspace(eigenvalues, eigenvectors):
    """
    Construct the choi matrix of the map from the positive and negative eigenspaces
    """
    eig_list = []
    for i in range(len(eigenvalues)):
        eig_list.append(eigenvalues[i] * (eigenvectors[i] @ eigenvectors[i].dag()))

    CP_map_choi = sum(eig_list)

    # CP_map_choi = (CP_map_choi + CP_map_choi.dag())/2
    
    return CP_map_choi


def decompose_into_extremal_CP_maps(choi_matrix_cp_map_adjoint):
    A = choi_matrix_cp_map_adjoint[:2, :2]
    C = choi_matrix_cp_map_adjoint[:2, 2:]
    C_dag = choi_matrix_cp_map_adjoint[2:, :2]
    B = choi_matrix_cp_map_adjoint[2:, 2:]

    # assert check_psd(I - A), "A > I error"
    # assert np.allclose(C.conjugate().T, C_dag, atol=1e-7), "C and C_dag are not conjugate transpose"
    # assert np.allclose(B, I - A, atol=1e-5), "B is not equal to I - A"

    sqrt_A = sqrtm(A)
    sqrt_B = sqrtm(B)

    # R = np.linalg.inv(sqrt_A) @ C @ np.linalg.inv(sqrt_B)
    R = pinv(sqrt_A) @ C @ pinv(sqrt_B)

    # print(np.linalg.norm(R)) # ---> Ensure <= 1 

    V, S, W_dag = np.linalg.svd(R)

    theta1 = np.arccos(np.clip(S[0], -1, 1))
    theta2 = np.arccos(np.clip(S[1], -1, 1))

    U1_diag = np.diag([np.exp(1j * theta1), np.exp(1j * theta2)])
    U2_diag = np.diag([np.exp(-1j * theta1), np.exp(-1j * theta2)])

    U1 = V @ U1_diag @ W_dag
    U2 = V @ U2_diag @ W_dag

    assert check_unitary(U1), "U1 is not unitary"
    assert check_unitary(U2), "U2 is not unitary"

    C1 = np.block([[A, sqrt_A @ U1 @ sqrt_B],
                   [sqrt_B @ U1.conj().T @ sqrt_A, B]])
    C2 = np.block([[A, sqrt_A @ U2 @ sqrt_B],
                   [sqrt_B @ U2.conj().T @ sqrt_A, B]])
    
    return C1, C2


def choi_to_map_in_kraus_rep(choi_matrix, skip_tp = False):
    """
    Convert a Choi matrix to Kraus operators.
    """
    dim = int(np.sqrt(choi_matrix.shape[0]))
    eigenvalues, eigenvectors = np.linalg.eigh(choi_matrix)
    kraus_operators = []

    for i, eigenvalue in enumerate(eigenvalues):
        if eigenvalue > 1e-10:  # Consider only positive eigenvalues
            vector = eigenvectors[:, i]
            psi = vector.reshape((dim, dim), order="F")
            K = np.sqrt(eigenvalue) * psi.T
            kraus_operators.append(K)

    if not skip_tp:
        assert check_TP(kraus_operators, tol=1e-4), "Kraus operators do not satisfy trace-preserving condition"

    return kraus_operators

def choi_to_superoperator(choi):
    """
    Convert the Choi rep to a superoperator.
    """
    kraus_operators = choi_to_map_in_kraus_rep(choi)
    dim = kraus_operators[0].shape[0]
    superoperator = np.zeros((dim**2, dim**2), dtype=complex)

    for K in kraus_operators:
        superoperator += np.kron(K.conjugate(), K)
    
    return superoperator

In [8]:
# Simulating one extremal CP map

def compute_rho_prime(kraus_operators, rho):
    """
    Compute the output density matrix from the input density matrix and the Kraus operators.
    """
    rho_prime = np.zeros_like(rho, dtype=complex)
    
    for K in kraus_operators:
        rho_prime += K @ rho @ K.conjugate().T
    
    return rho_prime

def T_matrix(kraus_operators):
    """
    Construct the T matrix from the map.
    """
    sigma_vec = [I, sigmax().full(), sigmay().full(), sigmaz().full()]
    dim = kraus_operators[0].shape[0]
    T = np.zeros((dim**2, dim**2), dtype=complex)

    for i in range(dim**2):
        for j in range(dim**2):
            T[i, j] = (1/2)*np.trace(sigma_vec[i] @ compute_rho_prime(kraus_operators, sigma_vec[j]))
    
    distortion_matrix = np.zeros((3, 3), dtype=np.float64)
    for i in range(3):
        for j in range(3):
            distortion_matrix[i, j] = np.real(T[i+1, j+1])
    
    distortion_matrix = np.real(distortion_matrix)
    
    shift_vector = T[:, 0][1:].reshape((3, 1))
    
    return T, distortion_matrix, shift_vector

def compute_unitary_and_angles(distortion_matrix, shift_vector):

    R1, T_prime, R2_T = np.linalg.svd(distortion_matrix, full_matrices=False)

    det_R1 = np.linalg.det(R1)
    det_R2 = np.linalg.det(R2_T)

    R1 = R1 / det_R1
    R2_T = R2_T / det_R2
    T_prime = T_prime / (det_R1 * det_R2)

    T_prime = np.real(np.diag(T_prime))

    nu = np.arccos(np.clip(T_prime[0, 0], -1, 1))
    mu = np.arccos(np.clip(T_prime[1, 1], -1, 1))

    assert np.isclose(T_prime[2, 2], np.cos(nu)*np.cos(mu), atol=1e-10), "T_prime[2] is not equal to cos(nu)*cos(mu)"

    t_prime = R2_T @ shift_vector

    assert np.isclose(t_prime[0, 0], 0, atol=1e-4), "t_prime[0] is not equal to 0"
    assert np.isclose(t_prime[1, 0], 0, atol=1e-4), "t_prime[1] is not equal to 0"
    # assert np.isclose(t_prime[2, 0], np.sin(mu)*np.sin(nu), atol=1e-7), "t_prime[2, 0] is not equal to sin(mu)*sin(nu)"

    return R1, R2_T.T, mu, nu


def so3_to_su2(R):
    sigma_vec = [sigmax().full(), sigmay().full(), sigmaz().full()]

    theta = np.arccos((1/2)*(np.trace(R) - 1))

    if np.isclose(theta, 0, atol=1e-10):
        return np.eye(2), None, None

    eigvals, eigvecs = np.linalg.eig(R)
    idx = np.argmin(np.abs(eigvals - 1))

    n = np.real(eigvecs[:, idx])
    n = n / np.linalg.norm(n)

    sigma_n = np.sum([n[i] * sigma_vec[i] for i in range(3)], axis=0)

    U = expm(-1j * theta/2 * sigma_n)

    assert check_unitary(U), "U is not unitary"

    return U, n, theta


In [9]:
# Build circuit

def build_circuit(U, V, mu, nu, time_step, initial_state = "0", last = True):
    """
    Build the circuit from the unitary and angles
    """
    alpha = (1/2)*(mu + nu)
    beta = (1/2)*(mu - nu)

    gamma_1 = (1/2)*(beta - alpha + np.pi/2)
    gamma_2 = (1/2)*(beta + alpha - np.pi/2)

    qubits = QuantumRegister(2, "q")
    cbits = ClassicalRegister(2, "c")
    circuit = QuantumCircuit(qubits, cbits)

    (data, ancilla) = qubits
    (c0, c1) = cbits

    # initial state
    if initial_state == "1":
        circuit.x(data)
    elif initial_state == "+":
        circuit.h(data)
    elif initial_state == "-":
        circuit.x(data)
        circuit.h(data)

    # for step in range(time_step):
    circuit.unitary(V, [data], label="V")
    circuit.ry(2*gamma_1, ancilla)
    circuit.cx(data, ancilla)
    circuit.ry(2*gamma_2, ancilla)

    circuit.measure(ancilla, c1)
    
    with circuit.if_test((c1, 1)):
        circuit.x(data)
        if not last:
            circuit.x(ancilla)
    
    circuit.unitary(U, [data], label="U")
    
    if last:
        circuit.measure(data, c0)
    
    return circuit

# Running the Circuit with Simulator/IBM device

def run_circuit(observable, circuit, sim = 'Yes', backend = None):
    if sim == 'Yes':
        sim_backend = AerSimulator()
        pm = generate_preset_pass_manager(backend=sim_backend, optimization_level=3)
        qc = pm.run(circuit)
        estimator = EstimatorV2(sim_backend)
        estimator.options.default_shots = 10000
        job = estimator.run([(qc, observable )])
        result = job.result()
    elif sim == 'No':
        pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
        isa_circuit = pm.run(circuit)

        mapped_obs = [obs.apply_layout(isa_circuit.layout) for obs in observable]

        with Session(backend=backend) as session:
            estimator = Estimator(mode=session)
            estimator.options.default_shots = 4096
            job = estimator.run([(isa_circuit, mapped_obs)])
            print("Job ID: ", job.job_id())
            result = job.result()

    return result[0].data.evs

In [10]:
def single_forward_step_run(dynamical_map):
    '''
    Returns the circuits for the single forward step of the dynamical map
    '''
    
    choi_matrix = construct_choi_matrix(dynamical_map)
    pos_eigenvalues, pos_eigenvectors, neg_eigenvalues, neg_eigenvectors = positive_and_negative_eigenspaces(choi_matrix)
    
    CP_map_choi_pos = construct_choi_matrix_from_eigenspace(pos_eigenvalues, pos_eigenvectors)

    assert check_choi_decomposition(choi_matrix, CP_map_choi_pos, 0), "Choi decomposition is not correct"

    choi_matrix_cp_map_adjoint = choi_adjointchoi_transform(CP_map_choi_pos)
    C1, C2 = decompose_into_extremal_CP_maps(choi_matrix_cp_map_adjoint)
    kraus_operators_C1 = choi_to_map_in_kraus_rep(C1)
    kraus_operators_C2 = choi_to_map_in_kraus_rep(C2)

    T_C1, distortion_matrix_C1, shift_vector_C1 = T_matrix(kraus_operators_C1)
    T_C2, distortion_matrix_C2, shift_vector_C2 = T_matrix(kraus_operators_C2)

    R11, R12, mu1, nu1 = compute_unitary_and_angles(distortion_matrix_C1, shift_vector_C1)
    R21, R22, mu2, nu2 = compute_unitary_and_angles(distortion_matrix_C2, shift_vector_C2)

    U1, n1, theta1 = so3_to_su2(R11)
    U2, n2, theta2 = so3_to_su2(R21)
    V1, n1, theta1 = so3_to_su2(R12)
    V2, n2, theta2 = so3_to_su2(R22)

    return U1, U2, V1, V2, mu1, mu2, nu1, nu2

def build_single_forward_step_circuit(U1, U2, V1, V2, mu1, mu2, nu1, nu2, time_step, initial_state = "0", last = True):
    circuit_1 = build_circuit(U1, V1, mu1, nu1, time_step, initial_state = initial_state, last = last)
    circuit_2 = build_circuit(U2, V2, mu2, nu2, time_step, initial_state = initial_state, last = last)

    return circuit_1, circuit_2

In [11]:
# Lindbladian to General Dynamical Map

def get_liouvillian_superoperator(omega, gamma, beta):

    # Pauli operators
    sz = sigmaz()
    sm = sigmam()
    sp = sigmap()

    H = 0.5 * omega * sz

    # Jump operators
    L1 = np.sqrt(gamma * np.exp(beta * omega)) * sm  # excitation
    L2 = np.sqrt(gamma) * sp                         # relaxation

    L = liouvillian(H, [L1, L2])
    return L


def get_dynamical_map(L, dt, coefficient = 1):
    """
    Get the dynamical map from the Liouvillian superoperator for time dt.
    """
    # Time evolution operator
    U = (coefficient * L * dt).expm()
    
    return U

In [12]:
omega, gamma, beta = 1.0, 1.0, 1.0

L = get_liouvillian_superoperator(omega, gamma, beta)
dt = 0.01
total_time = 0.2
time_steps = int(total_time/dt)
print(time_steps)

20


In [13]:
observable = SparsePauliOp.from_sparse_list([("Z", [0], -1/2), ("I", [0], 1/2)], num_qubits=2)

results = []

# U1, U2, V1, V2, mu1, mu2, nu1, nu2 = single_forward_step_run(dynamical_map)

for step in range(time_steps):
    dynamical_map = get_dynamical_map(L, dt*(step + 1))

    U1, U2, V1, V2, mu1, mu2, nu1, nu2 = single_forward_step_run(dynamical_map)

    circuit_1, circuit_2 = build_single_forward_step_circuit(U1, U2, V1, V2, mu1, mu2, nu1, nu2, step, initial_state = "1")
    result_1 = run_circuit(observable, circuit_1, sim="No", backend=backend)
    result_2 = run_circuit(observable, circuit_2, sim="No", backend=backend)

    result = (result_1 + result_2)/2
    results.append(result)
    print(f"Step {step} result: {result}")

Job ID:  d0khamjfbx30008wr0v0


RuntimeJobFailureError: "Unable to retrieve job result. Error code 1522; Job not valid. 'Dynamic circuits used in a V2 Estimator job. -- V2 Estimator does not support dynamic circuits. Remove control flow operations or convert the job to use V2 Sampler instead. -- https://ibm.biz/error_codes#1522'"

In [ ]:
import matplotlib.pyplot as plt

plt.plot(results, "-o", label="Initial state = |1>")
# plt.ylim(0.47, 0.55)
plt.xlabel("Time step")
plt.ylabel("Population of |1>")
plt.title("Expectation value of observable over time")
plt.show()

In [ ]:
np.save("./pop1_initial1_device.npy", np.array(results))